In [ ]:
# this Notebook uses a simple logit estimator - no frills!
# 
# 1) attempt: all the data but only most recent of duplicate content_id
# Version 7 - ROC AUC = 0.746 pafile.txt
#
# 2) attempt: last 10 content_id but only first of duplicates
# Version 8 - ROCAUC=0.716 pafile10.txt  (on train = 0.8239)
# Version 36  ROCAUC=0.658 rocauc106    (on train = 0.7662)
# Version 37  ROCAUC=0.715 better

#
# 3) attempt: last 20 content_id but only first of duplicates
# Version 10 - ROCAUC=0.651 pafile20.txt  (on train = 0.8062)
#
# 4) attempt: p=0.5 to check ROC AUC computation
# Version 12 ROCAUC=0.5
#
# 5) attempt: last 30 content_id but only first of duplicates
# Version 13 - ROCAUC= 0.742 pafile30.txt  (on train =0.8005)
#
# 6) attempt: last 40 content_id but only first of duplicates
# Version 14 - ROCAUC=0.746 pafile40.txt (on train = 0.7981)
#
# 7) attempt: last 50 content_id but only first of duplicates
# Version 15 - ROCAUC= 0.749 pafile50.txt (on train = 0.7964 )
#
# 8) attempt: last 60 content_id but only first of duplicates
# Version 16 - ROCAUC= 0.750 pafile60.txt (on train = 0.7952)
#
# 9) attempt: global variable for each n observations: 0.5, 0.6., 
# Version 17 - ROCAUC=0.5 size is 500K
#
# 10) attempt: last 70 content_id but only first of duplicates
# Version 19 - ROCAUC=0.751  pafile70.txt (on train = 0.7943)
#
# 11) attempt: last 80 content_id but only first of duplicates
# Version 20 - ROCAUC=0.752  pafile80.txt ()

# Version 22: chasing elusive scoring error pa60 ia80 - error
# Version 23: ia60 pa80 ROC = .752
# Version 24: with the files! but p=0.6 ROC=0.5 - worked
# Version 25: last 80 again - failed - trailing vbtab
# Version 26: last 80 again ROC = .752
# Version 27: last 70 again ROC = .751
# Version 28: this was 70 again - Oops! - maybe with a loop :-( ROC = .751
# version 29: "last90" is actually facets 80 +.02 for time: ROC = 0.663
# version 30: last90win ROC = 0.753  (on train = 0.7936) slot10=0.7904, 0.7916
# version 31: last100win ROC = 0.754 (on train = 0.7931)
# version 38: better100 optimized ROCAUC ROC = 0.754 (on train = 0.7895)
# version 32: last110win ROC = 0.754 (on train = 0.7927)
# version 33: last120win ROC = 0.754
# version 39: last130 ROC = 0.400 (plogit + ilogit)
# version 40: last130 ROC = 0.755 (plogit-ilogit)
# version 41: last140 ROC = 0.755
# version 42: last150 ROC = 0.754
# version 43: last135 ROC = 0.755
# version 44: try 0 person at 0.5 instead of 0.6791 ROC = 0.755
# version 45: only pafile135 ROC= 0.623
# version 47: only iafile135 ROC= 0.705
# version 48: < 0.67 -> 0.5 ROC = 0.697
# version 49: iafile135+item discrimination cut-off at 0.67 :-( roc=0.696
# version 50: iafile135+item discrimination roc= 0.756
# version 51: cut-off at probability 0.25 = 0.754
# version 52: iafile135+item discrimination + cut-off at 0.1 = 0.756
# version 53: ipvalueall ROC = .704
# version 54: ppvalueall ROC = .615 - suggests that person measures are about useless!
# version 55: i*pvalueall ROC = .733
# version 56: i+pvalueall/2 ROC = .737
# version 57: 2*i + pvalueall/3 ROC = .734
# version 58: 3*1 + pvalueall/4 ROC = .728
# version 59: i+p*2 pvalueall/3 ROC = .715
# version 60: i+pvaluel35/2 ROC = .744
# version 61: iafileall40 + pafile135-1 ROC =
# version 62: iafileall40 + pafile40x135 ROC =

In [ ]:
# do the housekeeping

import numpy as np
import pandas as pd
import bisect
import math
import numbers
import decimal
import riiideducation



In [ ]:
# global pafile

persons = pd.read_csv("../input/persons/persons.txt",sep="\t")
pafile = pd.read_csv("../input/pafile40x135/pafile40x135.txt",sep="\t")
iafile = pd.read_csv("../input/iafileall40/iafileall40.txt",sep="\t")

# pafile.iat[0,0] = 0.5

if pafile.iat[393656,0]!=393656:
    print ("bad pafile")
    exit
if iafile.iat[13523,0]!=13523:
    print ("bad iafile")
    exit    

print ("loaded")

# plogit = pafile['MEASURE'].tolist()
# getting length of list 
#length = len(plogit) 
#for i in range(length): 
#    print(i, plogit[i])
#    plogit[i] = plogit[i]+1
#    print(i, plogit[i])

# exit

In [ ]:
env = riiideducation.make_env()

In [ ]:
# compute a probability for each data line

def prob(user, content):
    userseq = bisect.bisect_left(persons.iloc[:,1],user)
    if persons.iat[userseq,1] != user: userseq=0
    # global plogit    
    plogit = pafile.iat[userseq,1]
    # plogit = 0
    # increments by .02 logits per response
    # print (userseq, plogit[userseq])
    # if userseq!=0: plogit[userseq] = plogit[userseq]+.02
    ilogit = iafile.iat[content,1]
    # idiscr = iafile.iat[content,2]
    # ilogit = 0
    # logit = plogit[userseq]-ilogit
    
    # rocauc
    logit = (plogit-ilogit) # * idiscr
    p= 1/(1+math.exp(-logit))
    # if p < 0.1: p=0.1
    # print(user,content, userseq,plogit[userseq],ilogit,logit,p)
    # print(user,content, userseq,plogit,ilogit,logit,p)
    # p = (plogit+(ilogit))/2
    return p   
    

In [ ]:
# go down the data lines

def mikes(df):
    user = df['user_id']
    content=df['content_id']
    if df['content_type_id'] != 0: content = 0
    p = prob (user, content)
    df['timestamp']=p
    return df['timestamp']


In [ ]:
# loop down the tests

iter_test = env.iter_test()
for (test_df, sample_prediction_df) in iter_test:
     test_df['timestamp'] = test_df.apply(mikes,axis=1)
     test_df=test_df.rename(columns = {'timestamp':'answered_correctly'})   
     env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])      
        
print ("done - yeah!")
